In [ ]:
import os
import json
import numpy as np
from qiskit import transpile
from disqco.circuits.cp_fraction import cp_fraction
from disqco.graphs.GCP_hypergraph import QuantumCircuitHyperGraph
from disqco.parti.FM.FM_main import *
from disqco.parti.FM.multilevel_FM import *

###############################################################################
# Set up JSON file for storing *all* iteration results (detailed data)
###############################################################################
detailed_filename = "benchmark_results_MLFM-R_CP_large_4-10part_linear.json"

if os.path.exists(detailed_filename):
    with open(detailed_filename, "r") as f:
        detailed_results = json.load(f)
else:
    detailed_results = []

###############################################################################
# Set up JSON file for *aggregated* results (mean cost/time)
###############################################################################
means_filename = "benchmark_means_MLFM-R_CP_large_4-10part_linear.json"



if os.path.exists(means_filename):
    with open(means_filename, "r") as f:
        mean_results = json.load(f)
else:
    mean_results = []

sizes = range(112, 257, 16)

num_partitions_list = [4,6,8]

for num_partitions in num_partitions_list:
    for i, num_qubits in enumerate(sizes):

        # Create an All-to-All network
        qpu_info = [int(num_qubits / num_partitions) + 1 for _ in range(num_partitions)]
        network = QuantumNetwork(qpu_info)
        
        iteration_data = []
        for iteration in range(5):
            
            # -------------------------
            # 1. Define circuit
            # -------------------------

            circuit = cp_fraction(num_qubits,num_qubits,fraction=0.5)

            circuit = transpile(circuit, basis_gates = ['u','cp'])
            base_graph = QuantumCircuitHyperGraph(circuit,group_gates=True)
            depth = base_graph.depth
            initial_assignment = set_initial_partitions(network,num_qubits, depth)
            
            # -------------------------
            # 2. MLFM-R
            # -------------------------
            num_levels = int(np.ceil(np.log2(num_qubits)))
            
            assignment_list_r, cost_list_r, time_list_r = MLFM_recursive_hetero(base_graph,
                                        initial_assignment,  
                                        qpu_info, 
                                        limit = num_qubits, 
                                        pass_list= [10]*(num_levels+1), 
                                        stochastic=True, 
                                        lock_nodes=False,
                                        log = False,
                                        add_initial = False,
                                        costs = {},
                                        network=network)
            
            total_time_r = sum(time_list_r)
            min_cost_r = min(cost_list_r)

            
            # -------------------------
            # 3. Store iteration-level results
            # -------------------------
            result_entry = {
                "num_qubits": num_qubits,
                "num_partitions": num_partitions,
                "iteration": iteration,
                "r_cost":  min_cost_r,
                "time_r": total_time_r,
            }
            
            detailed_results.append(result_entry)
            iteration_data.append(result_entry)
            
            with open(detailed_filename, "w") as f:
                json.dump(detailed_results, f, indent=2)
        # -------------------------
        # 4. Aggregate results
        r_cost_list = [x["r_cost"] for x in iteration_data]
        r_time_list = [x["time_r"] for x in iteration_data]
        

        mean_r_cost = float(np.mean(r_cost_list))
        mean_r_time = float(np.mean(r_time_list))
        
        print("=============================================")
        print(f"Finished 5 iterations for:")
        print(f"  # Qubits: {num_qubits}, # Partitions: {num_partitions}")
        print("Mean Costs:")

        print(f"  Recursive:{mean_r_cost:.3f}")
        print("Mean Times (s):")

        print(f"  Recursive:{mean_r_time:.3f}")
        print("=============================================")
        
        mean_entry = {
            "num_qubits": num_qubits,
            "num_partitions": num_partitions,
            "mean_r_cost": mean_r_cost,
            "mean_r_time": mean_r_time,
        }
        
        mean_results.append(mean_entry)
        
        with open(means_filename, "w") as f:
            json.dump(mean_results, f, indent=2)

print("Benchmarking completed. Detailed results saved to", detailed_filename)
print("Aggregated means saved to", means_filename)

Finished 5 iterations for:
  # Qubits: 112, # Partitions: 2
Mean Costs:
  Recursive:755.600
Mean Times (s):
  Recursive:1.814
Finished 5 iterations for:
  # Qubits: 128, # Partitions: 2
Mean Costs:
  Recursive:998.200
Mean Times (s):
  Recursive:2.567
Finished 5 iterations for:
  # Qubits: 144, # Partitions: 2
Mean Costs:
  Recursive:1261.400
Mean Times (s):
  Recursive:3.697
Finished 5 iterations for:
  # Qubits: 160, # Partitions: 2
Mean Costs:
  Recursive:1555.400
Mean Times (s):
  Recursive:4.781
Finished 5 iterations for:
  # Qubits: 176, # Partitions: 2
Mean Costs:
  Recursive:1882.400
Mean Times (s):
  Recursive:5.885
Finished 5 iterations for:
  # Qubits: 192, # Partitions: 2
Mean Costs:
  Recursive:2262.400
Mean Times (s):
  Recursive:7.463
Finished 5 iterations for:
  # Qubits: 208, # Partitions: 2
Mean Costs:
  Recursive:2658.800
Mean Times (s):
  Recursive:8.612
Finished 5 iterations for:
  # Qubits: 224, # Partitions: 2
Mean Costs:
  Recursive:3081.200
Mean Times (s):
  Re

In [1]:
import os
import json
import numpy as np
from qiskit import transpile
from disqco.circuits.cp_fraction import cp_fraction
from disqco.graphs.GCP_hypergraph import QuantumCircuitHyperGraph
from disqco.parti.FM.FM_main import *
from disqco.parti.FM.multilevel_FM import *
from disqco.graphs.quantum_network import QuantumNetwork, linear_coupling, grid_coupling

###############################################################################
# Set up JSON file for storing *all* iteration results (detailed data)
###############################################################################
detailed_filename = "benchmark_results_MLFM-R_CP_2-12_linear_0.1.json"

if os.path.exists(detailed_filename):
    with open(detailed_filename, "r") as f:
        detailed_results = json.load(f)
else:
    detailed_results = []

###############################################################################
# Set up JSON file for *aggregated* results (mean cost/time)
###############################################################################
means_filename = "benchmark_means_MLFM-R_CP_2-12_linear_0.1.json"



if os.path.exists(means_filename):
    with open(means_filename, "r") as f:
        mean_results = json.load(f)
else:
    mean_results = []

sizes = range(16, 97, 8)

fractions = [0.1]

# num_partitions_list = [4,6,8,10]

 

for fraction in fractions:
    for i, num_qubits in enumerate(sizes):
        num_partitions = 2 + i 

        # Create an All-to-All network
        qpu_info = [int(num_qubits / num_partitions) + 1 for _ in range(num_partitions)]
        coupling = linear_coupling(num_partitions)
        network = QuantumNetwork(qpu_info, coupling)
        
        iteration_data = []
        for iteration in range(5):
            
            # -------------------------
            # 1. Define circuit
            # -------------------------

            circuit = cp_fraction(num_qubits,num_qubits,fraction=fraction)

            circuit = transpile(circuit, basis_gates = ['u','cp'])
            base_graph = QuantumCircuitHyperGraph(circuit,group_gates=True)
            depth = base_graph.depth
            initial_assignment = set_initial_partitions(network,num_qubits, depth)

            two_qubit_gate_count = circuit.count_ops().get('cp', 0)
            
            # -------------------------
            # 2. MLFM-R
            # -------------------------
            num_levels = int(np.ceil(np.log2(num_qubits)))
            
            assignment_list_r, cost_list_r, time_list_r = MLFM_recursive_hetero(base_graph,
                                        initial_assignment,  
                                        qpu_info, 
                                        limit = num_qubits, 
                                        pass_list= [10]*(num_levels+1), 
                                        stochastic=True, 
                                        lock_nodes=False,
                                        log = False,
                                        add_initial = False,
                                        costs = {},
                                        network=network
                                        )
            
            total_time_r = sum(time_list_r)
            min_cost_r = min(cost_list_r)

            ebit_fraction = min_cost_r / two_qubit_gate_count if two_qubit_gate_count > 0 else 0
            
            # -------------------------
            # 3. Store iteration-level results
            # -------------------------
            result_entry = {
                "num_qubits": num_qubits,
                "num_partitions": num_partitions,
                "iteration": iteration,
                "fraction": fraction,
                "two_qubit_gate_count": two_qubit_gate_count,
                "r_cost":  min_cost_r,
                "time_r": total_time_r,
                "ebit_fraction": ebit_fraction,
            }
            
            detailed_results.append(result_entry)
            iteration_data.append(result_entry)
            
            with open(detailed_filename, "w") as f:
                json.dump(detailed_results, f, indent=2)
        # -------------------------
        # 4. Aggregate results
        r_cost_list = [x["r_cost"] for x in iteration_data]
        r_time_list = [x["time_r"] for x in iteration_data]
        gates_list = [x["two_qubit_gate_count"] for x in iteration_data]
        ebit_list = [x["ebit_fraction"] for x in iteration_data]
        

        mean_r_cost = float(np.mean(r_cost_list))
        mean_r_time = float(np.mean(r_time_list))
        mean_ebit_fraction = float(np.mean(ebit_list))
        mean_two_qubit_gate_count = float(np.mean(gates_list))
        
        print("=============================================")
        print(f"Finished 5 iterations for:")
        print(f"  # Qubits: {num_qubits}, # Partitions: {num_partitions} , fraction: {fraction}")
        print("Mean Costs:")

        print(f"  Recursive:{mean_r_cost:.3f}")
        print("Mean Times (s):")

        print(f"  Recursive:{mean_r_time:.3f}")
        print("=============================================")
        
        mean_entry = {
            "num_qubits": num_qubits,
            "num_partitions": num_partitions,
            "fraction": fraction,
            "mean_r_cost": mean_r_cost,
            "mean_r_time": mean_r_time,
            "mean_two_qubit_gate_count": mean_two_qubit_gate_count,
            "mean_ebit_fraction": mean_ebit_fraction,
            
            
        }
        
        mean_results.append(mean_entry)
        
        with open(means_filename, "w") as f:
            json.dump(mean_results, f, indent=2)

print("Benchmarking completed. Detailed results saved to", detailed_filename)
print("Aggregated means saved to", means_filename)

Finished 5 iterations for:
  # Qubits: 16, # Partitions: 2 , fraction: 0.1
Mean Costs:
  Recursive:0.000
Mean Times (s):
  Recursive:0.020
Finished 5 iterations for:
  # Qubits: 24, # Partitions: 3 , fraction: 0.1
Mean Costs:
  Recursive:2.800
Mean Times (s):
  Recursive:0.075
Finished 5 iterations for:
  # Qubits: 32, # Partitions: 4 , fraction: 0.1
Mean Costs:
  Recursive:11.800
Mean Times (s):
  Recursive:0.205
Finished 5 iterations for:
  # Qubits: 40, # Partitions: 5 , fraction: 0.1
Mean Costs:
  Recursive:36.000
Mean Times (s):
  Recursive:0.479
Finished 5 iterations for:
  # Qubits: 48, # Partitions: 6 , fraction: 0.1
Mean Costs:
  Recursive:61.200
Mean Times (s):
  Recursive:0.844
Finished 5 iterations for:
  # Qubits: 56, # Partitions: 7 , fraction: 0.1
Mean Costs:
  Recursive:123.800
Mean Times (s):
  Recursive:1.567
Finished 5 iterations for:
  # Qubits: 64, # Partitions: 8 , fraction: 0.1
Mean Costs:
  Recursive:202.000
Mean Times (s):
  Recursive:2.300
Finished 5 iteration

In [2]:
import os
import json
import numpy as np
from qiskit import transpile
from disqco.circuits.cp_fraction import cp_fraction
from disqco.graphs.GCP_hypergraph import QuantumCircuitHyperGraph
from disqco.parti.FM.FM_main import *
from disqco.parti.FM.multilevel_FM import *
from disqco.graphs.quantum_network import QuantumNetwork, linear_coupling, grid_coupling

###############################################################################
# Set up JSON file for storing *all* iteration results (detailed data)
###############################################################################
detailed_filename = "benchmark_results_MLFM-R_CP_2-12_part_grid_0.1.json"

if os.path.exists(detailed_filename):
    with open(detailed_filename, "r") as f:
        detailed_results = json.load(f)
else:
    detailed_results = []

###############################################################################
# Set up JSON file for *aggregated* results (mean cost/time)
###############################################################################
means_filename = "benchmark_means_MLFM-R_CP_2-12_part_grid_0.1.json"



if os.path.exists(means_filename):
    with open(means_filename, "r") as f:
        mean_results = json.load(f)
else:
    mean_results = []

sizes = range(16, 97, 8)

fractions = [0.1]

# num_partitions_list = [4,6,8,10]

 

for fraction in fractions:
    for i, num_qubits in enumerate(sizes):
        num_partitions = 2 + i 

        # Create an All-to-All network
        qpu_info = [int(num_qubits / num_partitions) + 1 for _ in range(num_partitions)]
        # coupling = linear_coupling(num_partitions)
        coupling = grid_coupling(num_partitions)
        network = QuantumNetwork(qpu_info, coupling)
        
        iteration_data = []
        for iteration in range(5):
            
            # -------------------------
            # 1. Define circuit
            # -------------------------

            circuit = cp_fraction(num_qubits,num_qubits,fraction=fraction)

            circuit = transpile(circuit, basis_gates = ['u','cp'])
            base_graph = QuantumCircuitHyperGraph(circuit,group_gates=True)
            depth = base_graph.depth
            initial_assignment = set_initial_partitions(network,num_qubits, depth)

            two_qubit_gate_count = circuit.count_ops().get('cp', 0)
            
            # -------------------------
            # 2. MLFM-R
            # -------------------------
            num_levels = int(np.ceil(np.log2(num_qubits)))
            
            assignment_list_r, cost_list_r, time_list_r = MLFM_recursive_hetero(base_graph,
                                        initial_assignment,  
                                        qpu_info, 
                                        limit = num_qubits, 
                                        pass_list= [10]*(num_levels+1), 
                                        stochastic=True, 
                                        lock_nodes=False,
                                        log = False,
                                        add_initial = False,
                                        costs = {},
                                        network=network
                                        )
            
            total_time_r = sum(time_list_r)
            min_cost_r = min(cost_list_r)

            ebit_fraction = min_cost_r / two_qubit_gate_count if two_qubit_gate_count > 0 else 0
            
            # -------------------------
            # 3. Store iteration-level results
            # -------------------------
            result_entry = {
                "num_qubits": num_qubits,
                "num_partitions": num_partitions,
                "iteration": iteration,
                "fraction": fraction,
                "two_qubit_gate_count": two_qubit_gate_count,
                "r_cost":  min_cost_r,
                "time_r": total_time_r,
                "ebit_fraction": ebit_fraction,
            }
            
            detailed_results.append(result_entry)
            iteration_data.append(result_entry)
            
            with open(detailed_filename, "w") as f:
                json.dump(detailed_results, f, indent=2)
        # -------------------------
        # 4. Aggregate results
        r_cost_list = [x["r_cost"] for x in iteration_data]
        r_time_list = [x["time_r"] for x in iteration_data]
        gates_list = [x["two_qubit_gate_count"] for x in iteration_data]
        ebit_list = [x["ebit_fraction"] for x in iteration_data]
        

        mean_r_cost = float(np.mean(r_cost_list))
        mean_r_time = float(np.mean(r_time_list))
        mean_ebit_fraction = float(np.mean(ebit_list))
        mean_two_qubit_gate_count = float(np.mean(gates_list))
        
        print("=============================================")
        print(f"Finished 5 iterations for:")
        print(f"  # Qubits: {num_qubits}, # Partitions: {num_partitions} , fraction: {fraction}")
        print("Mean Costs:")

        print(f"  Recursive:{mean_r_cost:.3f}")
        print("Mean Times (s):")

        print(f"  Recursive:{mean_r_time:.3f}")
        print("=============================================")
        
        mean_entry = {
            "num_qubits": num_qubits,
            "num_partitions": num_partitions,
            "fraction": fraction,
            "mean_r_cost": mean_r_cost,
            "mean_r_time": mean_r_time,
            "mean_two_qubit_gate_count": mean_two_qubit_gate_count,
            "mean_ebit_fraction": mean_ebit_fraction,
            
            
        }
        
        mean_results.append(mean_entry)
        
        with open(means_filename, "w") as f:
            json.dump(mean_results, f, indent=2)

print("Benchmarking completed. Detailed results saved to", detailed_filename)
print("Aggregated means saved to", means_filename)

Finished 5 iterations for:
  # Qubits: 16, # Partitions: 2 , fraction: 0.1
Mean Costs:
  Recursive:0.200
Mean Times (s):
  Recursive:0.020
Finished 5 iterations for:
  # Qubits: 24, # Partitions: 3 , fraction: 0.1
Mean Costs:
  Recursive:4.000
Mean Times (s):
  Recursive:0.092
Finished 5 iterations for:
  # Qubits: 32, # Partitions: 4 , fraction: 0.1
Mean Costs:
  Recursive:12.600
Mean Times (s):
  Recursive:0.230
Finished 5 iterations for:
  # Qubits: 40, # Partitions: 5 , fraction: 0.1
Mean Costs:
  Recursive:30.200
Mean Times (s):
  Recursive:0.479
Finished 5 iterations for:
  # Qubits: 48, # Partitions: 6 , fraction: 0.1
Mean Costs:
  Recursive:50.800
Mean Times (s):
  Recursive:0.815
Finished 5 iterations for:
  # Qubits: 56, # Partitions: 7 , fraction: 0.1
Mean Costs:
  Recursive:89.600
Mean Times (s):
  Recursive:1.437
Finished 5 iterations for:
  # Qubits: 64, # Partitions: 8 , fraction: 0.1
Mean Costs:
  Recursive:128.600
Mean Times (s):
  Recursive:2.127
Finished 5 iterations

In [3]:
import os
import json
import numpy as np
from qiskit import transpile
from disqco.circuits.cp_fraction import cp_fraction
from disqco.graphs.GCP_hypergraph import QuantumCircuitHyperGraph
from disqco.parti.FM.FM_main import *
from disqco.parti.FM.multilevel_FM import *
from disqco.graphs.quantum_network import QuantumNetwork, linear_coupling, grid_coupling, random_coupling

###############################################################################
# Set up JSON file for storing *all* iteration results (detailed data)
###############################################################################
detailed_filename = "benchmark_results_MLFM-R_CP_2-12_part_random_0.1.json"

if os.path.exists(detailed_filename):
    with open(detailed_filename, "r") as f:
        detailed_results = json.load(f)
else:
    detailed_results = []

###############################################################################
# Set up JSON file for *aggregated* results (mean cost/time)
###############################################################################
means_filename = "benchmark_means_MLFM-R_CP_2-12_part_random_0.1.json"



if os.path.exists(means_filename):
    with open(means_filename, "r") as f:
        mean_results = json.load(f)
else:
    mean_results = []

sizes = range(16, 97, 8)

fractions = [0.1]

# num_partitions_list = [4,6,8,10]

 

for fraction in fractions:
    for i, num_qubits in enumerate(sizes):
        num_partitions = 2 + i 

        # Create an All-to-All network
        qpu_info = [int(num_qubits / num_partitions) + 1 for _ in range(num_partitions)]
        # coupling = linear_coupling(num_partitions)
        coupling = random_coupling(num_partitions, 0.5)
        network = QuantumNetwork(qpu_info, coupling)
        
        iteration_data = []
        for iteration in range(5):
            
            # -------------------------
            # 1. Define circuit
            # -------------------------

            circuit = cp_fraction(num_qubits,num_qubits,fraction=fraction)

            circuit = transpile(circuit, basis_gates = ['u','cp'])
            base_graph = QuantumCircuitHyperGraph(circuit,group_gates=True)
            depth = base_graph.depth
            initial_assignment = set_initial_partitions(network,num_qubits, depth)

            two_qubit_gate_count = circuit.count_ops().get('cp', 0)
            
            # -------------------------
            # 2. MLFM-R
            # -------------------------
            num_levels = int(np.ceil(np.log2(num_qubits)))
            
            assignment_list_r, cost_list_r, time_list_r = MLFM_recursive_hetero(base_graph,
                                        initial_assignment,  
                                        qpu_info, 
                                        limit = num_qubits, 
                                        pass_list= [10]*(num_levels+1), 
                                        stochastic=True, 
                                        lock_nodes=False,
                                        log = False,
                                        add_initial = False,
                                        costs = {},
                                        network=network
                                        )
            
            total_time_r = sum(time_list_r)
            min_cost_r = min(cost_list_r)

            ebit_fraction = min_cost_r / two_qubit_gate_count if two_qubit_gate_count > 0 else 0
            
            # -------------------------
            # 3. Store iteration-level results
            # -------------------------
            result_entry = {
                "num_qubits": num_qubits,
                "num_partitions": num_partitions,
                "iteration": iteration,
                "fraction": fraction,
                "two_qubit_gate_count": two_qubit_gate_count,
                "r_cost":  min_cost_r,
                "time_r": total_time_r,
                "ebit_fraction": ebit_fraction,
            }
            
            detailed_results.append(result_entry)
            iteration_data.append(result_entry)
            
            with open(detailed_filename, "w") as f:
                json.dump(detailed_results, f, indent=2)
        # -------------------------
        # 4. Aggregate results
        r_cost_list = [x["r_cost"] for x in iteration_data]
        r_time_list = [x["time_r"] for x in iteration_data]
        gates_list = [x["two_qubit_gate_count"] for x in iteration_data]
        ebit_list = [x["ebit_fraction"] for x in iteration_data]
        

        mean_r_cost = float(np.mean(r_cost_list))
        mean_r_time = float(np.mean(r_time_list))
        mean_ebit_fraction = float(np.mean(ebit_list))
        mean_two_qubit_gate_count = float(np.mean(gates_list))
        
        print("=============================================")
        print(f"Finished 5 iterations for:")
        print(f"  # Qubits: {num_qubits}, # Partitions: {num_partitions} , fraction: {fraction}")
        print("Mean Costs:")

        print(f"  Recursive:{mean_r_cost:.3f}")
        print("Mean Times (s):")

        print(f"  Recursive:{mean_r_time:.3f}")
        print("=============================================")
        
        mean_entry = {
            "num_qubits": num_qubits,
            "num_partitions": num_partitions,
            "fraction": fraction,
            "mean_r_cost": mean_r_cost,
            "mean_r_time": mean_r_time,
            "mean_two_qubit_gate_count": mean_two_qubit_gate_count,
            "mean_ebit_fraction": mean_ebit_fraction,
            
            
        }
        
        mean_results.append(mean_entry)
        
        with open(means_filename, "w") as f:
            json.dump(mean_results, f, indent=2)

print("Benchmarking completed. Detailed results saved to", detailed_filename)
print("Aggregated means saved to", means_filename)

Finished 5 iterations for:
  # Qubits: 16, # Partitions: 2 , fraction: 0.1
Mean Costs:
  Recursive:0.600
Mean Times (s):
  Recursive:0.022
Finished 5 iterations for:
  # Qubits: 24, # Partitions: 3 , fraction: 0.1
Mean Costs:
  Recursive:3.600
Mean Times (s):
  Recursive:0.081
Finished 5 iterations for:
  # Qubits: 32, # Partitions: 4 , fraction: 0.1
Mean Costs:
  Recursive:11.400
Mean Times (s):
  Recursive:0.216
Finished 5 iterations for:
  # Qubits: 40, # Partitions: 5 , fraction: 0.1
Mean Costs:
  Recursive:27.800
Mean Times (s):
  Recursive:0.452
Finished 5 iterations for:
  # Qubits: 48, # Partitions: 6 , fraction: 0.1
Mean Costs:
  Recursive:53.600
Mean Times (s):
  Recursive:0.870
Finished 5 iterations for:
  # Qubits: 56, # Partitions: 7 , fraction: 0.1
Mean Costs:
  Recursive:80.200
Mean Times (s):
  Recursive:1.459
Finished 5 iterations for:
  # Qubits: 64, # Partitions: 8 , fraction: 0.1
Mean Costs:
  Recursive:115.600
Mean Times (s):
  Recursive:2.394
Finished 5 iterations

In [12]:
import os
import json
import numpy as np
from qiskit import transpile
from disqco.circuits.cp_fraction import cp_fraction
from disqco.graphs.GCP_hypergraph import QuantumCircuitHyperGraph
from disqco.parti.FM.FM_main import *
from disqco.parti.FM.multilevel_FM import *

###############################################################################
# Set up JSON file for storing *all* iteration results (detailed data)
###############################################################################
detailed_filename = "benchmark_results_MLFM-R_CP_large_4-10part_linear.json"

if os.path.exists(detailed_filename):
    with open(detailed_filename, "r") as f:
        detailed_results = json.load(f)
else:
    detailed_results = []

###############################################################################
# Set up JSON file for *aggregated* results (mean cost/time)
###############################################################################
means_filename = "benchmark_means_MLFM-R_CP_large_4-10part_linear.json"



if os.path.exists(means_filename):
    with open(means_filename, "r") as f:
        mean_results = json.load(f)
else:
    mean_results = []

sizes = range(112, 257, 16)

num_partitions_list = [8,10]

for num_partitions in num_partitions_list:
    for i, num_qubits in enumerate(sizes):

        # Create an All-to-All network
        qpu_info = [int(num_qubits / num_partitions) + 1 for _ in range(num_partitions)]
        coupling = linear_coupling(num_partitions)
        network = QuantumNetwork(qpu_info, coupling)
        
        iteration_data = []
        for iteration in range(5):
            
            # -------------------------
            # 1. Define circuit
            # -------------------------

            circuit = cp_fraction(num_qubits,num_qubits,fraction=0.5)

            circuit = transpile(circuit, basis_gates = ['u','cp'])
            base_graph = QuantumCircuitHyperGraph(circuit,group_gates=True)
            depth = base_graph.depth
            initial_assignment = set_initial_partitions(network,num_qubits, depth)
            
            # -------------------------
            # 2. MLFM-R
            # -------------------------
            num_levels = int(np.ceil(np.log2(num_qubits)))
            
            assignment_list_r, cost_list_r, time_list_r = MLFM_recursive_hetero(base_graph,
                                        initial_assignment,  
                                        qpu_info, 
                                        limit = num_qubits, 
                                        pass_list= [10]*(num_levels+1), 
                                        stochastic=True, 
                                        lock_nodes=False,
                                        log = False,
                                        add_initial = False,
                                        costs = {},
                                        network=network)
            
            total_time_r = sum(time_list_r)
            min_cost_r = min(cost_list_r)

            
            # -------------------------
            # 3. Store iteration-level results
            # -------------------------
            result_entry = {
                "num_qubits": num_qubits,
                "num_partitions": num_partitions,
                "iteration": iteration,
                "r_cost":  min_cost_r,
                "time_r": total_time_r,
            }
            
            detailed_results.append(result_entry)
            iteration_data.append(result_entry)
            
            with open(detailed_filename, "w") as f:
                json.dump(detailed_results, f, indent=2)
        # -------------------------
        # 4. Aggregate results
        r_cost_list = [x["r_cost"] for x in iteration_data]
        r_time_list = [x["time_r"] for x in iteration_data]
        

        mean_r_cost = float(np.mean(r_cost_list))
        mean_r_time = float(np.mean(r_time_list))
        
        print("=============================================")
        print(f"Finished 5 iterations for:")
        print(f"  # Qubits: {num_qubits}, # Partitions: {num_partitions}")
        print("Mean Costs:")

        print(f"  Recursive:{mean_r_cost:.3f}")
        print("Mean Times (s):")

        print(f"  Recursive:{mean_r_time:.3f}")
        print("=============================================")
        
        mean_entry = {
            "num_qubits": num_qubits,
            "num_partitions": num_partitions,
            "mean_r_cost": mean_r_cost,
            "mean_r_time": mean_r_time,
        }
        
        mean_results.append(mean_entry)
        
        with open(means_filename, "w") as f:
            json.dump(mean_results, f, indent=2)

print("Benchmarking completed. Detailed results saved to", detailed_filename)
print("Aggregated means saved to", means_filename)

Finished 5 iterations for:
  # Qubits: 112, # Partitions: 8
Mean Costs:
  Recursive:3860.000
Mean Times (s):
  Recursive:32.569
Finished 5 iterations for:
  # Qubits: 128, # Partitions: 8
Mean Costs:
  Recursive:4979.800
Mean Times (s):
  Recursive:45.961
Finished 5 iterations for:
  # Qubits: 144, # Partitions: 8
Mean Costs:
  Recursive:6607.200
Mean Times (s):
  Recursive:63.058
Finished 5 iterations for:
  # Qubits: 160, # Partitions: 8
Mean Costs:
  Recursive:8030.800
Mean Times (s):
  Recursive:82.851
Finished 5 iterations for:
  # Qubits: 176, # Partitions: 8
Mean Costs:
  Recursive:9923.400
Mean Times (s):
  Recursive:111.008
Finished 5 iterations for:
  # Qubits: 192, # Partitions: 8
Mean Costs:
  Recursive:11582.000
Mean Times (s):
  Recursive:144.162
Finished 5 iterations for:
  # Qubits: 208, # Partitions: 8
Mean Costs:
  Recursive:14032.000
Mean Times (s):
  Recursive:182.251
Finished 5 iterations for:
  # Qubits: 224, # Partitions: 8
Mean Costs:
  Recursive:16102.200
Mean 

In [13]:
import os
import json
import numpy as np
from qiskit import transpile
from disqco.circuits.cp_fraction import cp_fraction
from disqco.graphs.GCP_hypergraph import QuantumCircuitHyperGraph
from disqco.parti.FM.FM_main import *
from disqco.parti.FM.multilevel_FM import *

###############################################################################
# Set up JSON file for storing *all* iteration results (detailed data)
###############################################################################
detailed_filename = "benchmark_results_MLFM-R_CP_large_4-10part_grid.json"

if os.path.exists(detailed_filename):
    with open(detailed_filename, "r") as f:
        detailed_results = json.load(f)
else:
    detailed_results = []

###############################################################################
# Set up JSON file for *aggregated* results (mean cost/time)
###############################################################################
means_filename = "benchmark_means_MLFM-R_CP_large_4-10part_grid.json"



if os.path.exists(means_filename):
    with open(means_filename, "r") as f:
        mean_results = json.load(f)
else:
    mean_results = []

sizes = range(112, 257, 16)

num_partitions_list = [4,6,8,10]

for num_partitions in num_partitions_list:
    for i, num_qubits in enumerate(sizes):

        # Create an All-to-All network
        qpu_info = [int(num_qubits / num_partitions) + 1 for _ in range(num_partitions)]
        coupling = grid_coupling(num_partitions)
        network = QuantumNetwork(qpu_info, coupling)
        
        iteration_data = []
        for iteration in range(5):
            
            # -------------------------
            # 1. Define circuit
            # -------------------------

            circuit = cp_fraction(num_qubits,num_qubits,fraction=0.5)

            circuit = transpile(circuit, basis_gates = ['u','cp'])
            base_graph = QuantumCircuitHyperGraph(circuit,group_gates=True)
            depth = base_graph.depth
            initial_assignment = set_initial_partitions(network,num_qubits, depth)
            
            # -------------------------
            # 2. MLFM-R
            # -------------------------
            num_levels = int(np.ceil(np.log2(num_qubits)))
            
            assignment_list_r, cost_list_r, time_list_r = MLFM_recursive_hetero(base_graph,
                                        initial_assignment,  
                                        qpu_info, 
                                        limit = num_qubits, 
                                        pass_list= [10]*(num_levels+1), 
                                        stochastic=True, 
                                        lock_nodes=False,
                                        log = False,
                                        add_initial = False,
                                        costs = {},
                                        network=network)
            
            total_time_r = sum(time_list_r)
            min_cost_r = min(cost_list_r)

            
            # -------------------------
            # 3. Store iteration-level results
            # -------------------------
            result_entry = {
                "num_qubits": num_qubits,
                "num_partitions": num_partitions,
                "iteration": iteration,
                "r_cost":  min_cost_r,
                "time_r": total_time_r,
            }
            
            detailed_results.append(result_entry)
            iteration_data.append(result_entry)
            
            with open(detailed_filename, "w") as f:
                json.dump(detailed_results, f, indent=2)
        # -------------------------
        # 4. Aggregate results
        r_cost_list = [x["r_cost"] for x in iteration_data]
        r_time_list = [x["time_r"] for x in iteration_data]
        

        mean_r_cost = float(np.mean(r_cost_list))
        mean_r_time = float(np.mean(r_time_list))
        
        print("=============================================")
        print(f"Finished 5 iterations for:")
        print(f"  # Qubits: {num_qubits}, # Partitions: {num_partitions}")
        print("Mean Costs:")

        print(f"  Recursive:{mean_r_cost:.3f}")
        print("Mean Times (s):")

        print(f"  Recursive:{mean_r_time:.3f}")
        print("=============================================")
        
        mean_entry = {
            "num_qubits": num_qubits,
            "num_partitions": num_partitions,
            "mean_r_cost": mean_r_cost,
            "mean_r_time": mean_r_time,
        }
        
        mean_results.append(mean_entry)
        
        with open(means_filename, "w") as f:
            json.dump(mean_results, f, indent=2)

print("Benchmarking completed. Detailed results saved to", detailed_filename)
print("Aggregated means saved to", means_filename)

Finished 5 iterations for:
  # Qubits: 112, # Partitions: 4
Mean Costs:
  Recursive:1389.800
Mean Times (s):
  Recursive:10.817
Finished 5 iterations for:
  # Qubits: 128, # Partitions: 4
Mean Costs:
  Recursive:1837.800
Mean Times (s):
  Recursive:15.085
Finished 5 iterations for:
  # Qubits: 144, # Partitions: 4
Mean Costs:
  Recursive:2316.000
Mean Times (s):
  Recursive:20.669
Finished 5 iterations for:
  # Qubits: 160, # Partitions: 4
Mean Costs:
  Recursive:2897.200
Mean Times (s):
  Recursive:26.522
Finished 5 iterations for:
  # Qubits: 176, # Partitions: 4
Mean Costs:
  Recursive:3482.600
Mean Times (s):
  Recursive:33.281
Finished 5 iterations for:
  # Qubits: 192, # Partitions: 4
Mean Costs:
  Recursive:4159.200
Mean Times (s):
  Recursive:40.851
Finished 5 iterations for:
  # Qubits: 208, # Partitions: 4
Mean Costs:
  Recursive:4914.800
Mean Times (s):
  Recursive:49.832
Finished 5 iterations for:
  # Qubits: 224, # Partitions: 4
Mean Costs:
  Recursive:5681.600
Mean Times 

KeyboardInterrupt: 

In [ ]:
import os
import json
import numpy as np
from qiskit import transpile
from disqco.circuits.cp_fraction import cp_fraction
from disqco.graphs.GCP_hypergraph import QuantumCircuitHyperGraph
from disqco.parti.FM.FM_main import *
from disqco.parti.FM.multilevel_FM import *

###############################################################################
# Set up JSON file for storing *all* iteration results (detailed data)
###############################################################################
detailed_filename = "benchmark_results_MLFM-R_CP_large_4-10part_random.json"

if os.path.exists(detailed_filename):
    with open(detailed_filename, "r") as f:
        detailed_results = json.load(f)
else:
    detailed_results = []

###############################################################################
# Set up JSON file for *aggregated* results (mean cost/time)
###############################################################################
means_filename = "benchmark_means_MLFM-R_CP_large_4-10part_random.json"



if os.path.exists(means_filename):
    with open(means_filename, "r") as f:
        mean_results = json.load(f)
else:
    mean_results = []

sizes = range(112, 257, 16)

num_partitions_list = [4,6,8,10]

for num_partitions in num_partitions_list:
    for i, num_qubits in enumerate(sizes):

        # Create an All-to-All network
        qpu_info = [int(num_qubits / num_partitions) + 1 for _ in range(num_partitions)]
        coupling = random_coupling(num_partitions, p=0.5)
        network = QuantumNetwork(qpu_info, coupling)
        
        iteration_data = []
        for iteration in range(5):
            
            # -------------------------
            # 1. Define circuit
            # -------------------------

            circuit = cp_fraction(num_qubits,num_qubits,fraction=0.5)

            circuit = transpile(circuit, basis_gates = ['u','cp'])
            base_graph = QuantumCircuitHyperGraph(circuit,group_gates=True)
            depth = base_graph.depth
            initial_assignment = set_initial_partitions(network,num_qubits, depth)
            
            # -------------------------
            # 2. MLFM-R
            # -------------------------
            num_levels = int(np.ceil(np.log2(num_qubits)))
            
            assignment_list_r, cost_list_r, time_list_r = MLFM_recursive_hetero(base_graph,
                                        initial_assignment,  
                                        qpu_info, 
                                        limit = num_qubits, 
                                        pass_list= [10]*(num_levels+1), 
                                        stochastic=True, 
                                        lock_nodes=False,
                                        log = False,
                                        add_initial = False,
                                        costs = {},
                                        network=network)
            
            total_time_r = sum(time_list_r)
            min_cost_r = min(cost_list_r)

            
            # -------------------------
            # 3. Store iteration-level results
            # -------------------------
            result_entry = {
                "num_qubits": num_qubits,
                "num_partitions": num_partitions,
                "iteration": iteration,
                "r_cost":  min_cost_r,
                "time_r": total_time_r,
            }
            
            detailed_results.append(result_entry)
            iteration_data.append(result_entry)
            
            with open(detailed_filename, "w") as f:
                json.dump(detailed_results, f, indent=2)
        # -------------------------
        # 4. Aggregate results
        r_cost_list = [x["r_cost"] for x in iteration_data]
        r_time_list = [x["time_r"] for x in iteration_data]
        

        mean_r_cost = float(np.mean(r_cost_list))
        mean_r_time = float(np.mean(r_time_list))
        
        print("=============================================")
        print(f"Finished 5 iterations for:")
        print(f"  # Qubits: {num_qubits}, # Partitions: {num_partitions}")
        print("Mean Costs:")

        print(f"  Recursive:{mean_r_cost:.3f}")
        print("Mean Times (s):")

        print(f"  Recursive:{mean_r_time:.3f}")
        print("=============================================")
        
        mean_entry = {
            "num_qubits": num_qubits,
            "num_partitions": num_partitions,
            "mean_r_cost": mean_r_cost,
            "mean_r_time": mean_r_time,
        }
        
        mean_results.append(mean_entry)
        
        with open(means_filename, "w") as f:
            json.dump(mean_results, f, indent=2)

print("Benchmarking completed. Detailed results saved to", detailed_filename)
print("Aggregated means saved to", means_filename)

In [ ]:
import os
import json
import numpy as np
from qiskit import transpile
from qiskit.circuit.library import QFT, QuantumVolume
from disqco.circuits.QAOA import QAOA_random

###############################################################################
# Set up JSON file for storing *all* iteration results (detailed data)
###############################################################################

circuit_type = "QAOA"
# circuit_type = "QFT"
# circuit_type = "QV"

detailed_filename = f"benchmark_results_MLFM-R_{circuit_type}_2-4part.json"


if os.path.exists(detailed_filename):
    with open(detailed_filename, "r") as f:
        detailed_results = json.load(f)
else:
    detailed_results = []

###############################################################################
# Set up JSON file for *aggregated* results (mean cost/time)
###############################################################################
means_filename = f"benchmark_means_MLFM-R_{circuit_type}_2-4part.json"

if os.path.exists(means_filename):
    with open(means_filename, "r") as f:
        mean_results = json.load(f)
else:
    mean_results = []

sizes = range(16, 97, 8)

num_partitions_list = [2,4]

for num_partitions in num_partitions_list:
    for i, num_qubits in enumerate(sizes):

        # Create an All-to-All network
        qpu_info = [int(num_qubits / num_partitions) + 1 for _ in range(num_partitions)]
        network = QuantumNetwork(qpu_info)
        
        iteration_data = []
        for iteration in range(5):
            
            # -------------------------
            # 1. Define/redefine circuit
            # -------------------------
            if circuit_type == "QAOA":
                circuit = QAOA_random(num_qubits,prob=0.5, reps=1)
            elif circuit_type == "QFT":
                circuit = QFT(num_qubits,do_swaps=False)
            elif circuit_type == "QV":
                circuit = QuantumVolume(num_qubits,num_qubits)
            else:
                raise ValueError(f"Unknown circuit type: {circuit_type}")
            
            circuit = transpile(circuit, basis_gates = ['u','cp'])

            base_graph = QuantumCircuitHyperGraph(circuit,group_gates=True)
            depth = base_graph.depth
            initial_assignment = set_initial_partitions(network, num_qubits, depth)
            
            # -------------------------
            # 2. MLFM-R
            # -------------------------
            num_levels = int(np.ceil(np.log2(num_qubits)))
            
            assignment_list_r, cost_list_r, time_list_r = MLFM_recursive(base_graph,
                                        initial_assignment,  
                                        qpu_info, 
                                        limit = num_qubits, 
                                        pass_list= [10]*(num_levels+1), 
                                        stochastic=True, 
                                        lock_nodes=False,
                                        log = False,
                                        add_initial = False,
                                        costs = None)
            
            total_time_r = sum(time_list_r)
            min_cost_r = min(cost_list_r)

            # -------------------------
            # 3. Store iteration-level results
            # -------------------------
            result_entry = {
                "num_qubits": num_qubits,
                "num_partitions": num_partitions,
                "iteration": iteration,
                "r_cost":  min_cost_r,
                "time_r": total_time_r,
            }
            
            detailed_results.append(result_entry)
            iteration_data.append(result_entry)
            
            # Update detailed JSON right away
            with open(detailed_filename, "w") as f:
                json.dump(detailed_results, f, indent=2)
        
        # ---------------------------------------------------------------------
        # After 5 iterations, compute the means and log them
        # ---------------------------------------------------------------------

        r_cost_list = [x["r_cost"] for x in iteration_data]
        r_time_list = [x["time_r"] for x in iteration_data]
        
        mean_r_cost = float(np.mean(r_cost_list))
        mean_r_time = float(np.mean(r_time_list))
        
        # Print to console for quick logging
        print("=============================================")
        print(f"Finished 5 iterations for:")
        print(f"  # Qubits: {num_qubits}, # Partitions: {num_partitions}")
        print("Mean Costs:")

        print(f"  Recursive:{mean_r_cost:.3f}")
        print("Mean Times (s):")

        print(f"  Recursive:{mean_r_time:.3f}")
        print("=============================================")
        
        # Store the aggregated means in a separate JSON
        mean_entry = {
            "num_qubits": num_qubits,
            "num_partitions": num_partitions,
            "mean_r_cost": mean_r_cost,
            "mean_r_time": mean_r_time,
        }
        
        mean_results.append(mean_entry)
        
        # Update the means JSON file
        with open(means_filename, "w") as f:
            json.dump(mean_results, f, indent=2)

print("Benchmarking completed. Detailed results saved to", detailed_filename)
print("Aggregated means saved to", means_filename)

Finished 5 iterations for:
  # Qubits: 112, # Partitions: 4
Mean Costs:
  Recursive:1212.200
Mean Times (s):
  Recursive:15.817
Finished 5 iterations for:
  # Qubits: 128, # Partitions: 4
Mean Costs:
  Recursive:1592.200
Mean Times (s):
  Recursive:21.744
Finished 5 iterations for:
  # Qubits: 144, # Partitions: 4
Mean Costs:
  Recursive:2013.600
Mean Times (s):
  Recursive:30.931
Finished 5 iterations for:
  # Qubits: 160, # Partitions: 4
Mean Costs:
  Recursive:2487.000
Mean Times (s):
  Recursive:39.881
Finished 5 iterations for:
  # Qubits: 176, # Partitions: 4
Mean Costs:
  Recursive:3023.200
Mean Times (s):
  Recursive:50.767
Finished 5 iterations for:
  # Qubits: 192, # Partitions: 4
Mean Costs:
  Recursive:3607.600
Mean Times (s):
  Recursive:61.211
Finished 5 iterations for:
  # Qubits: 208, # Partitions: 4
Mean Costs:
  Recursive:4252.800
Mean Times (s):
  Recursive:72.635
Finished 5 iterations for:
  # Qubits: 224, # Partitions: 4
Mean Costs:
  Recursive:4944.400
Mean Times 